* importing modules

In [ ]:
from scripts import imports
from scripts import visualization
import numpy as np
import pandas as pd
import json
import os

In [48]:
import numpy as np
import pandas as pd
import json
import os

def read_json(path:str = "config.json"):
    """
    Reading information from json file. Used to extract
    the parameters from the config.json.
    path : str -> path of the json file
    """
    with open('config.json') as config:
        config_f = json.load(config)
    return config_f

config_vals = read_json()


def list_datasets(path:str):
    """
    Returning list of all the csv files in a folder with their complete path
    path : str -> path of the folder
    """

    raw_list = os.listdir(path)
    csv_files = [i for i in raw_list if i.split(".")[1] == "csv"]
    print(f"{len(csv_files)} datasets found in {path}r_data:")

    for i in csv_files:
        print(i)
    
    return [f"{path}r_data\\{i}" for i in csv_files]

csv_files = list_datasets(f"{config_vals["output_folder"]}\\r_data")

8 datasets found in output\r_datar_data:
motif_tp2_[2025-11-27_20-15-00]_baseline_values.csv
motif_tp1_[2025-11-27_20-15-00]_baseline_values.csv
motif_tp4_[2025-11-27_20-15-00]_baseline_values.csv
motif_tpall_[2025-11-27_19-55-53]_baseline_values.csv
all_tpall_[2025-11-27_17-05-32]_baseline_values.csv
all_tp4_[2025-11-27_18-49-50]_baseline_values.csv
all_tp1_[2025-11-27_18-49-50]_baseline_values.csv
all_tp2_[2025-11-27_18-49-50]_baseline_values.csv
